In [ ]:
# Install required packages
!pip install streamlit pyngrok -q

# Write the Streamlit app to a .py file
app_code = """
import streamlit as st
import random

st.set_page_config(page_title="Quantum Chat App", page_icon="💬")

st.title("💬 Quantum Chat Simulation")
st.markdown("Secure communication between Alice 🧕 and Bob 👨‍💻 using quantum principles.")

st.sidebar.title("🔧 Simulation Settings")
key_length = st.sidebar.slider("Key Length", 8, 128, 32)

# Generate random quantum key
def generate_key(length):
    return [random.randint(0, 1) for _ in range(length)]

alice_key = generate_key(key_length)
bob_key = alice_key.copy()

# Introduce optional eavesdropping
eavesdrop = st.sidebar.checkbox("Simulate Eavesdropper (Eve) 🕵️")
if eavesdrop:
    for i in range(len(bob_key)):
        if random.random() < 0.25:
            bob_key[i] = 1 - bob_key[i]  # Flip bit

# Shared secret message
message = st.text_input("💬 Enter your message (Embassy A ➡️ Embassy B)", "Quantum encrypted ops at dawn.")

# XOR Encrypt
def xor_encrypt(msg, key):
    binary = ''.join(format(ord(c), '08b') for c in msg)
    enc = [int(b)^key[i%len(key)] for i, b in enumerate(binary)]
    return enc

def xor_decrypt(enc_bits, key):
    decrypted = [str(b^key[i%len(key)]) for i, b in enumerate(enc_bits)]
    chars = [chr(int(''.join(decrypted[i:i+8]), 2)) for i in range(0, len(decrypted), 8)]
    return ''.join(chars)

encrypted_bits = xor_encrypt(message, alice_key)
decrypted_message = xor_decrypt(encrypted_bits, bob_key)

# Display info
st.subheader("🔐 Quantum Key")
st.text(f"Alice's Key: {''.join(map(str, alice_key))}")
st.text(f"Bob's Key:   {''.join(map(str, bob_key))}")

error_count = sum(a != b for a, b in zip(alice_key, bob_key))
error_rate = (error_count / len(alice_key)) * 100

st.markdown(f"**Error Rate:** `{error_rate:.2f}%`")
if error_rate > 25:
    st.error("🚨 High error rate! Possible eavesdropping detected.")
else:
    st.success("✅ Channel is likely secure.")

st.subheader("📡 Transmission")
st.code(f"Encrypted Bits: {''.join(map(str, encrypted_bits))}")
st.markdown(f"**📥 Decrypted Message at Bob's Side:** `{decrypted_message}`")
"""

with open("quantum_chat_app.py", "w") as f:
    f.write(app_code)

# Launch the app using ngrok
from pyngrok import ngrok
import threading

def run_app():
    !streamlit run quantum_chat_app.py --server.port 8501

threading.Thread(target=run_app).start()
!ngrok config add-authtoken 2uBedLAyaJGvIop4Yq5AlZTlVke_6q5RsE8ADTdL2yX62vMs6

public_url = ngrok.connect(8501)
print(f"🔗 Your app is live at: {public_url}")


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🔗 Your app is live at: NgrokTunnel: "https://77c1-35-224-134-238.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
# Save the Streamlit app as a .py file
from pathlib import Path

app_code = """
import streamlit as st
import numpy as np
import random
import matplotlib.pyplot as plt

st.set_page_config(page_title="Quantum Chat - BB84", layout="centered")

st.title("📡 Quantum Chat App - BB84 QKD Protocol")
st.markdown("Secure communication using quantum key distribution (BB84 Protocol)")

# Settings
num_bits = st.slider("Select number of qubits:", 10, 100, 30)
secret_message = st.text_input("💬 Enter your secret message (from Alice to Bob)", "Meet at 9 PM")
simulate_eve = st.checkbox("🕵️ Simulate Eavesdropper (Eve)")

# 1. Alice generates random bits and bases
alice_bits = np.random.randint(2, size=num_bits)
alice_bases = np.random.choice(["Z", "X"], size=num_bits)

# 2. Eve intercepts and measures qubits
if simulate_eve:
    eve_bases = np.random.choice(["Z", "X"], size=num_bits)

    # Eve measures each bit
    def eve_measure(bit, basis_sender, basis_eve):
        if basis_sender == basis_eve:
            return bit
        else:
            return np.random.randint(2)

    eve_results = np.array([eve_measure(alice_bits[i], alice_bases[i], eve_bases[i]) for i in range(num_bits)])
    # Eve sends new bits to Bob
    transmitted_bits = eve_results
    transmitted_bases = eve_bases
else:
    # No eavesdropper — Alice's original bits are transmitted
    transmitted_bits = alice_bits
    transmitted_bases = alice_bases

# 3. Bob chooses random bases
bob_bases = np.random.choice(["Z", "X"], size=num_bits)

# 4. Bob measures based on received qubits and his basis
def bob_measure(bit, basis_sender, basis_bob):
    if basis_sender == basis_bob:
        return bit
    else:
        return np.random.randint(2)

bob_results = np.array([bob_measure(transmitted_bits[i], transmitted_bases[i], bob_bases[i]) for i in range(num_bits)])

# 5. Shared key: only bits where Alice and Bob bases match
matching_indices = np.where(alice_bases == bob_bases)[0]
shared_key = alice_bits[matching_indices]
bob_key = bob_results[matching_indices]

# 6. Calculate error rate
errors = np.sum(shared_key != bob_key)
error_rate = (errors / len(shared_key)) * 100 if len(shared_key) > 0 else 0
eavesdrop_detected = error_rate > 25

# 7. Encrypt and decrypt the message using XOR
def xor_encrypt_decrypt(message, key):
    bin_msg = ''.join(format(ord(c), '08b') for c in message)
    key_repeated = np.resize(key, len(bin_msg))
    encrypted = [int(b) ^ int(k) for b, k in zip(bin_msg, key_repeated)]
    return encrypted

def bits_to_text(bits):
    chars = [chr(int(''.join(map(str, bits[i:i+8])), 2)) for i in range(0, len(bits), 8)]
    return ''.join(chars)

# Encrypt with Bob's key
encrypted_bits = xor_encrypt_decrypt(secret_message, bob_key)
decrypted_bits = xor_encrypt_decrypt(bits_to_text(encrypted_bits), bob_key)
decrypted_message = bits_to_text(decrypted_bits)

# Display results
st.subheader("🔐 Communication Summary")
st.markdown(f"**Alice's Original Message:** `{secret_message}`")
st.markdown(f"**Encrypted Binary:** `{''.join(map(str, encrypted_bits))}`")
st.markdown(f"**Bob's Decrypted Message:** `{decrypted_message}`")
st.markdown(f"**Error Rate:** `{error_rate:.2f}%`")

if eavesdrop_detected:
    st.error("⚠️ Eavesdropping Detected! Abort communication.")
else:
    st.success("✅ Secure Channel Established!")

# Optional: Visualize shared key
st.subheader("📊 Shared Key Visualization")
fig, ax = plt.subplots()
ax.plot(shared_key, 'o-', label="Alice's Key")
ax.plot(bob_key, 'x--', label="Bob's Key")
ax.set_xlabel("Bit Index")
ax.set_ylabel("Bit Value")
ax.set_title("Key Comparison")
ax.legend()
st.pyplot(fig)

"""

# Save to file
Path("quantum_chat_app.py").write_text(app_code)


3551

In [ ]:
# Start Streamlit app with ngrok
from pyngrok import ngrok
import threading
import time

def run():
    !streamlit run quantum_chat_app.py

# Run app in a separate thread
thread = threading.Thread(target=run)
thread.start()

# Wait for Streamlit to boot up
time.sleep(5)
!ngrok config add-authtoken 2uBedLAyaJGvIop4Yq5AlZTlVke_6q5RsE8ADTdL2yX62vMs6x
# Open the public URL via ngrok
public_url = ngrok.connect(8501)
print(f"🌐 Your Quantum Chat App is running at: {public_url}")


ModuleNotFoundError: No module named 'pyngrok'